In [1]:
# import data
import numpy as np
import json

# Change .json filename to select animal ID number

# import standard experiment data (XS) and metadata (S_labels)
with open('data/pyData/2705_Data_S_Notch14_1.json') as f:
    data = json.load(f)
pxx = np.array(data[0])
pxx = np.log(pxx)
a, b, c = pxx.shape
XS = pxx.reshape(a*b, c).T
S_labels = data[1]

# Train data ONLY on standard experiments from day 1: X, S_labels where subject contains 1
# stimON = 1 vs nonstim = 0 (baseline AND stimOFF)

days = np.zeros(len(S_labels['allWindows']['subject']))
for k in range(0, len(S_labels['allWindows']['subject'])):
    days[k] = int(S_labels['allWindows']['subject'][k][6])-1

numDay = int(np.ceil(max(days)))+1
duration = 539
    
X_arr = np.zeros((numDay, duration), dtype = int)
Y_arr = np.zeros((numDay, duration), dtype = int)
    
for k in range(0, len(S_labels['allWindows']['subject'])):
    d = int(S_labels['allWindows']['subject'][k][6])-1
    w = S_labels['allWindows']['windowID'][k]-1
    if w < duration: 
        X_arr[d, w] = k
        Y_arr[d, w] = S_labels['allWindows']['stimOn'][k]

X_list = X_arr.reshape(numDay*duration, 1)
Y_list = Y_arr.reshape(numDay*duration, 1)

X_all = XS
Y_all = np.array(S_labels['allWindows']['stimOn'])

In [2]:
from sklearn.decomposition import FastICA
from sklearn import svm
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_curve, auc

components = np.logspace(1, 2.7783, 15, dtype = int)

roc_auc = np.zeros((numDay, (numDay-1), len(components)))
best = np.zeros((numDay, 2))
#testID = np.arange(0, Y_all.shape[0], dtype = 'int')

outer_cv = KFold(n_splits = numDay, shuffle = True)
inner_cv = KFold(n_splits = (numDay-1), shuffle = True)
t = 0
#    inner_cv = KFold(n_splits = 8)
for subtrain, test in outer_cv.split(range(0, numDay)):
    v = 0
    for train, validate in inner_cv.split(subtrain):
        c = 0
        X_inner_trainingset = X_all[X_arr[subtrain[train], ].reshape((numDay-2)*duration), ]
        Y_inner_trainingset = Y_arr[subtrain[train], ].reshape((numDay-2)*duration)
        X_validate = X_all[X_arr[subtrain[validate], ], ].reshape(duration, 600)
        Y_validate = Y_all[X_arr[subtrain[validate]]].reshape(duration)

        for C in components:   
            print(test, subtrain[validate], subtrain[train], C)
            
            ica = FastICA(n_components = C, max_iter = 5000,tol = 0.0001) #tol = 0.001
            X_inner_train = ica.fit_transform(X_inner_trainingset) #pull components from ica fit transformation
            X_inner_test = ica.transform(X_validate)
            
            clf = svm.SVC(kernel = 'linear', class_weight = 'balanced', probability = True)
            y_inner_score = clf.fit(X_inner_train, Y_inner_trainingset).decision_function(X_inner_test)
            fpr, tpr, _ = roc_curve(Y_validate, y_inner_score)
            roc_auc[t, v, c] = auc(fpr, tpr)

            c += 1
        v += 1
    
    best[t, 0] = int(np.argmax(np.mean(roc_auc[t, :, :], axis = 0)))
    print('Best components | %0.0f' % (components[int(best[t, 0])]))
    
    X_outer_trainingset = X_all[X_arr[subtrain, ].reshape((numDay-1)*duration), ]
    Y_outer_trainingset = Y_arr[subtrain, ].reshape((numDay-1)*duration)
    X_test = X_all[X_arr[test, ], ].reshape(duration, 600)
    Y_test = Y_all[X_arr[test]].reshape(duration)
    
    
    ica = FastICA(n_components = components[int(best[t, 0])], max_iter = 5000,tol = 0.0001) #tol = 0.001
    X_outer_train = ica.fit_transform(X_outer_trainingset) #pull components from ica fit transformation
    X_outer_test = ica.transform(X_test)
    
    clf = svm.SVC(kernel = 'linear', class_weight = 'balanced', probability = True)
    y_outer_score = clf.fit(X_outer_train, Y_outer_trainingset).decision_function(X_outer_test)
    fpr, tpr, _ = roc_curve(Y_test, y_outer_score)
    best[t, 1] = auc(fpr, tpr)
    print('8x1 ROC AUC | %0.2f' % (best[t, 1]*100))
    t += 1
    
for i in range(0,numDay): best[i, 0] = components[int(best[i, 0])]
    
np.savetxt('nestedROC_AUC.txt', roc_auc.reshape(numDay*(numDay-1),len(components)), delimiter='\t')
np.savetxt('nestedTestROC_AUC.txt', best, delimiter='\t')

[4] [0] [1 2 3 4 5 6 7] 10
[4] [0] [1 2 3 4 5 6 7] 13
[4] [0] [1 2 3 4 5 6 7] 17
[4] [0] [1 2 3 4 5 6 7] 23
[4] [0] [1 2 3 4 5 6 7] 32
[4] [0] [1 2 3 4 5 6 7] 42
[4] [0] [1 2 3 4 5 6 7] 57
[4] [0] [1 2 3 4 5 6 7] 76
[4] [0] [1 2 3 4 5 6 7] 102
[4] [0] [1 2 3 4 5 6 7] 137
[4] [0] [1 2 3 4 5 6 7] 183
[4] [0] [1 2 3 4 5 6 7] 245
[4] [0] [1 2 3 4 5 6 7] 328
[4] [0] [1 2 3 4 5 6 7] 440


C:\Users\Bradley Barth\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\decomposition\fastica_.py:118: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


[4] [0] [1 2 3 4 5 6 7] 588
[4] [3] [0 1 2 4 5 6 7] 10
[4] [3] [0 1 2 4 5 6 7] 13
[4] [3] [0 1 2 4 5 6 7] 17
[4] [3] [0 1 2 4 5 6 7] 23
[4] [3] [0 1 2 4 5 6 7] 32
[4] [3] [0 1 2 4 5 6 7] 42
[4] [3] [0 1 2 4 5 6 7] 57
[4] [3] [0 1 2 4 5 6 7] 76
[4] [3] [0 1 2 4 5 6 7] 102
[4] [3] [0 1 2 4 5 6 7] 137
[4] [3] [0 1 2 4 5 6 7] 183
[4] [3] [0 1 2 4 5 6 7] 245
[4] [3] [0 1 2 4 5 6 7] 328
[4] [3] [0 1 2 4 5 6 7] 440
[4] [3] [0 1 2 4 5 6 7] 588
[4] [5] [0 1 2 3 4 6 7] 10
[4] [5] [0 1 2 3 4 6 7] 13
[4] [5] [0 1 2 3 4 6 7] 17
[4] [5] [0 1 2 3 4 6 7] 23
[4] [5] [0 1 2 3 4 6 7] 32
[4] [5] [0 1 2 3 4 6 7] 42
[4] [5] [0 1 2 3 4 6 7] 57
[4] [5] [0 1 2 3 4 6 7] 76
[4] [5] [0 1 2 3 4 6 7] 102
[4] [5] [0 1 2 3 4 6 7] 137
[4] [5] [0 1 2 3 4 6 7] 183
[4] [5] [0 1 2 3 4 6 7] 245
[4] [5] [0 1 2 3 4 6 7] 328
[4] [5] [0 1 2 3 4 6 7] 440
[4] [5] [0 1 2 3 4 6 7] 588
[4] [1] [0 2 3 4 5 6 7] 10
[4] [1] [0 2 3 4 5 6 7] 13
[4] [1] [0 2 3 4 5 6 7] 17
[4] [1] [0 2 3 4 5 6 7] 23
[4] [1] [0 2 3 4 5 6 7] 32
[4] [1] [0 2 

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

#for k in range(9):
#    plt.scatter([k+1]*len(roc_auc[k, :, int(best[t, 0])]), roc_auc[k, :, int(best[t, 0])], c = '0.8')

ax0 = plt.subplot2grid((1, numDay+1), (0, 0), colspan = numDay)
plt.plot(range(1,numDay+1), best[:, 1], linestyle = '-', c = '0.8')
plt.plot(range(1,numDay+1), best[:, 1], 'ko')

plt.ylim([0, 1])
plt.ylabel('ROC AUC')
plt.xlim([0.5, numDay+0.5])
plt.xlabel('Day')
plt.xticks(np.arange(1, numDay+1))

ax0.spines['top'].set_visible(False)
ax0.spines['right'].set_visible(False)

if np.std(best[:, 0]) == 0:
    plt.text(0.75, 0.875, 'Mean AUC = %0.2f' % (np.mean(best[:, 1])), horizontalalignment = 'left')
    plt.text(0.75, 0.95, 'Components | %0.0f' % (components[int(best[0, 0])]), horizontalalignment = 'left')
else:
    for d in range(0, numDay):
        plt.text(d+1, 0.875, '%0.2f' % (best[d, 1]), horizontalalignment = 'center')
        plt.text(d+1, 0.95, '%0.0f' % (components[int(best[d, 0])]), horizontalalignment = 'center')

ax1 = plt.subplot2grid((1, numDay+1), (0, numDay), colspan = 1)
plt.boxplot(best[:, 1], widths = 0.8)

plt.ylim([0, 1])
plt.yticks([-1], (''))
plt.xticks([0], (''))

ax1.spines['top'].set_visible(False)
ax1.spines['bottom'].set_visible(False)
ax1.spines['left'].set_visible(False)
ax1.spines['right'].set_visible(False)

plt.show()

In [ ]:
for t in range(0, 9):
    for k in range(len(components)):
        plt.scatter([k+1]*len(roc_auc[t, :, k]), roc_auc[t, :, k])

    plt.boxplot(roc_auc[t, :, :])
    plt.ylim([0, 1])
    plt.ylabel('ROC AUC')
    plt.xticks(np.arange(1, 1+len(components)), components)
    plt.xlabel('Number of components')
    plt.title('Test day | %0.0f' %(t+1))
    plt.text(0.75, 0.875, 'Mean AUC = %0.2f' % (best[t, 1]*100), horizontalalignment = 'left')
    plt.text(0.75, 0.95, 'Components | %0.0f' % (components[int(best[t, 0])]), horizontalalignment = 'left')
    plt.show()

# (1) split scores and labels into training and testing groups
# (2) plot data as boxplots, and difference of the weights between neuron and muscle

from sklearn.model_selection import StratifiedKFold
from sklearn import svm
from sklearn.metrics import roc_curve, auc
from sklearn.decomposition import FastICA
import matplotlib.pyplot as plt

components = [20] #np.logspace(1, np.log10(200), 5, dtype=int) #[10, 25, 50, 100, 150, 200, 250]
testID = np.arange(0, Y_all.shape[0], dtype = 'int')
    
n = 5 #10 # number of folds
F = len(components) # number of sets of folds
testAcc = np.zeros((F, n))
fpr = dict()
tpr = dict()

performance = list(range(len(components)))
roc_auc = np.zeros((F, n))

outer_cv = KFold(n_splits = 9)
for train, test in outer_cv.split(range(0, 9)):
    skf = StratifiedKFold(n_splits = n, shuffle = True)   
    t = 0
    
    X_trainingset = X_all[X_arr[train, ].reshape((numDay-1)*duration)]
    Y_trainingset = Y_all[Y_arr[train, ].reshape((numDay-1)*duration)]
    
    for train_index, test_index in skf.split(X_trainingset, Y_trainingset):
        ica = FastICA(n_components = C, max_iter = 5000,tol = 0.0001) #tol = 0.001
        scores = ica.fit_transform(X_all) #pull components from ica fit transformation
        scoreClass = scores[testID]
        X_train, X_test = scoreClass[train_index], scoreClass[test_index]
        y_train, y_test = np.array(Y_all)[train_index], np.array(Y_all)[test_index]
        clf = svm.SVC(kernel = 'linear', class_weight = 'balanced', probability = True)
        y_score = clf.fit(X_train, y_train).decision_function(X_test)
        fpr, tpr, _ = roc_curve(y_test, y_score)
        roc_auc[fld, t] = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw = 1.5)
        t += 1
         
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.text(0.975, 0.05, 'Mean AUC = %0.2f' % (np.mean(roc_auc[fld, ])*100), horizontalalignment = 'right')
    plt.text(0.975, 0.15, 'Components | %0.0f' % (C), horizontalalignment = 'right')
    plt.show()
    
    performance[fld] = np.mean(roc_auc[fld, ])*100
    
std = np.std(roc_auc, axis = 1)*100

ax = plt.subplot(1, 1, 1)
ax.set_xscale("log", nonposx='clip')
plt.errorbar(components, performance, yerr = std, linestyle = '', marker = '.', ecolor = 'b', mfc = 'k')
plt.ylabel('Performance (ROC AUC)')
plt.xlabel('Number of Components')
plt.xticks(components, components)
plt.ylim([0, 100])
plt.show()

ax = plt.subplot(1, 1, 1)
ax.set_xscale("log", nonposx='clip')
std = np.std(roc_auc, axis = 1)*100
plt.errorbar(components, performance, yerr = std, linestyle = '', marker = '.', ecolor = 'b', mfc = 'k')
plt.ylabel('Performance (ROC AUC)')
plt.xlabel('Number of Components')
plt.xticks(components, components)
plt.ylim([50, 100])
plt.show()